## Imports

In [1]:
import numpy as np
import ipsuite as ips
import mace_models

import ase
import ase.constraints
import ase.calculators

2024-05-08 11:26:44,166 (DEBUG): Welcome to IPS - the Interatomic Potential Suite!


In [2]:
model = mace_models.load()


        You're using the MACE-MP-0 model. The model is released under the MIT license.
        Note:
        If you are using this model, please cite the relevant paper for the Materials Project,
        any paper associated with the MACE model, and also the following:
        - MACE-Universal by Yuan Chiang, 2023, Hugging Face, Revision e5ebd9b,
            DOI: 10.57967/hf/1202, URL: https://huggingface.co/cyrusyc/mace-universal
        - Matbench Discovery by Janosh Riebesell, Rhys EA Goodall, Philipp Benner, Yuan Chiang,
            Alpha A Lee, Anubhav Jain, Kristin A Persson, 2023, arXiv:2308.14920
        - https://arxiv.org/abs/2401.00096
           


## Inital Data Generation

In [3]:
with ips.Project(remove_existing_graph=True, automatic_node_names=True) as project:
    with project.group("IDG") as group:  # Initial Data Generation

        # Generate a single water
        water = ips.configuration_generation.SmilesToAtoms(
            smiles="O", name="SMILES-Water"
        )  # H20

        # Place molecules in box
        packmol = ips.configuration_generation.Packmol(
            data=[water.atoms],
            count=[100],
            density=997,
        )

        # Perform geometry optimization on the newly generated molecules
        geoopt = ips.calculators.ASEGeoOpt(
            data=packmol.atoms,
            model=model,
            optimizer="FIRE",
            run_kwargs={"fmax": 0.5},
        )

# MLP MD

In [4]:
thermostat = ips.calculators.LangevinThermostat(
            temperature=300, friction=0.01, time_step=0.5
        )
model2 = ips.calculators.TorchD3(xc="pbe", data=None, device="cpu", damping="bj", cutoff=8, abc=False, cnthr=8, dtype="float32")

with project:
    with project.group("MLP_MD") as group:  # MLP Warer Molecular Dynamics
        
        mix = ips.calculators.MixCalculator(data=geoopt.atoms, calculators=[model, model2], methods="sum")

        md = ips.calculators.ASEMD(
            data=geoopt.atoms,
            data_id=-1,
            model=mix,
            thermostat=thermostat,
            steps=200_000,  # vorher: 500_000
            sampling_rate=1,
        )

In [5]:
project.build()

Running DVC command: 'stage add --name MLP_MD_ASEMD --force ...': 100%|███████████████████████████████████████| 6/6 [00:00<00:00,  8.72it/s]
